In [0]:
# sample input file:
import pandas as pd
import numpy as np
#from pandas.io.json import json_normalize
import pickle
import urllib.request
import bisect
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


def get_predictions(df):
  df1 = df.copy() 
  user_labels = 'https://github.com/bhragu/Alset/blob/master/weights/user_labels.pkl?raw=true'  
  urllib.request.urlretrieve(user_labels, 'user_labels.pkl') 

  activity_labels = 'https://github.com/bhragu/Alset/blob/master/weights/activity_labels.pkl?raw=true'  
  urllib.request.urlretrieve(activity_labels, 'activity_labels.pkl') 

  ensemble01 = 'https://github.com/bhragu/Alset/blob/master/weights/ensemble01.pkl?raw=true'  
  urllib.request.urlretrieve(ensemble01, 'ensemble01.pkl')  

  ensemble11 = 'https://github.com/bhragu/Alset/blob/master/weights/ensemble11.pkl?raw=true'  
  urllib.request.urlretrieve(ensemble11, 'ensemble11.pkl') 

  with open('activity_labels.pkl','rb') as model:
      le =pickle.load(model)
  df['activityKey'] = df['activityKey'].map(lambda s: 'other' if s not in le.classes_ else s)
  le_classes = le.classes_.tolist()
  bisect.insort_left(le_classes, 'other')
  le.classes_ = le_classes
  df['activityKey'] = le.transform(df['activityKey'])
  with open('user_labels.pkl','rb') as model:
      le =pickle.load(model)
  df['userKey'] = df['userKey'].map(lambda s: 'other' if s not in le.classes_ else s)
  le_classes = le.classes_.tolist()
  bisect.insort_left(le_classes, 'other')
  le.classes_ = le_classes
  df['userKey'] = le.transform(df['userKey'])
  with open('ensemble01.pkl','rb') as model:
      model =pickle.load(model)    
  predictions = model.predict(df)
  df['predictions'] = predictions
  result = df[df['predictions']==0]
  df = df[df['predictions']==1]
  df = df.drop(['predictions'], axis = 1)
  if df.shape[0]==0:  
      result = result.drop(['activityKey','userKey',],axis=1)  
      df2 = df1.merge(result, left_index=True, right_index= True)
      df2 = df2[['activityKey','userKey','predictions']]
      return df2
  with open('ensemble11.pkl','rb') as model:
      model =pickle.load(model)
  predictions = model.predict(df)
  df['predictions'] = predictions
  df['predictions'] = df['predictions'].replace(1,2)
  df['predictions'] = df['predictions'].replace(0,1)
  result = result.append(df)
  result = result.drop(['activityKey','userKey',],axis=1)
  df2 = df1.merge(result, left_index=True, right_index= True)
  df2 = df2[['activityKey','userKey','predictions']]
  return df2

data = pd.read_json("input.json")
output = get_predictions(data)
output.head()
output.to_json('result.json')
output.to_json('results.json')